# Decision Tree: Error over training size and validation curve

## Wine Quality Dataset
This dataset contains instances for red and white wine samples.
The inputs include objective tests (e.g. PH values) and the output is based on sensory data
(median of at least 3 evaluations made by wine experts). Each expert graded the wine quality 
between 0 (very bad) and 10 (very excellent).

The two datasets are related to red and white variants of the Portuguese "Vinho Verde" wine.
For more details, consult: http://www.vinhoverde.pt/en/ or the reference [Cortez et al., 2009].
Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables 
are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

In [ ]:
import pandas as pd
df = pd.read_csv('../Lecture_6-AppliedMachineLearning/data/winequality-white.csv', sep=';')
df.info()

We will keep a part of the dataset as a test set:

In [ ]:
# Prepare X and y...
features = list(df.columns[:-1])
X = df[features]
y = df["quality"]

import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Learning curve

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn import tree
clf = tree.DecisionTreeClassifier()

train_sizes, train_scores, valid_scores = learning_curve(clf, X, y, train_sizes=[500, 1000, 2000], cv=2)

In [ ]:
train_sizes

In [ ]:
train_scores

In [ ]:
valid_scores

### Plotting learning curve

Function from http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html#sphx-glr-auto-examples-model-selection-plot-learning-curve-py

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score (1- mean accuracy)")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores = 1 - train_scores
    test_scores  = 1 - test_scores
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score (1- mean accuracy)")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score (1- mean accuracy)")

    plt.legend(loc="best")
    return plt

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

plot_learning_curve(tree.DecisionTreeClassifier(), "Learning Curve", X, y)

## Do we have high bias, or high variance?

***

### Validation curve

In [ ]:
from sklearn.model_selection import validation_curve

train_scores, valid_scores = validation_curve(tree.DecisionTreeClassifier(),
                                              X, y, "max_depth", [None, 1000, 100, 50, 40, 30, 20, 10, 5, 2], cv=2)
train_scores

In [ ]:
valid_scores

## Plotting validation curve

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import validation_curve

def plot_validation_curve(estimator, title, X, y, param_name, param_range, cv=None,
                        n_jobs=1):
    plt.figure()
    plt.title(title)
    plt.xlabel(param_name)
    plt.ylabel("Score (mean accuracy)")
    plt.ylim(0.0, 1.1)
    train_scores, test_scores = validation_curve(
        estimator, X, y, param_name, param_range, cv=cv, n_jobs=n_jobs)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(param_range, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(param_range, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(param_range, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(param_range, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
plot_validation_curve(tree.DecisionTreeClassifier(), "Validation curve", X, y,
                      "max_depth", np.arange(30, 0, -1))